# Demo of `create_climpred_data`

This demo demonstrates how you setup your raw output from a climate model to match `climpred`'s expectations.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import climpred

In [2]:
from climpred.create_climpred_data import load_hindcast, climpred_preprocess_internal

Assuming your raw model output is stored in multiple files per member and initialization, `load_hindcast` is a nice wrapper function designed for the output format of `MPI-ESM` to aggregated all hindcast output into one file as expected by `climpred`.

The basic idea is to look over the output of all members and concatinate, then loop over all initializations and concatinate. Before concatination, it is important to align the `time` dimension.

To reduce the data size, use the `preprocess` function wisely in combination with `climpred_preprocess_internal`, e.g. additionally extracting only a certain region or only few variables for a multi-variable input file as in MPI-ESM standard output.

In [3]:
load_hindcast??

Object `load_hindcast` not found.


# `intake-esm` for cmorized output

In case you have access to cmorized output of CMIP experiments, consider using `intake-esm`. With the `preprocess` function you can align the `time` dimension of the output. Finally, `climpred_preprocess_post` only renames.

In [ ]:
from climpred.create_climpred_data import climpred_preprocess_post, climpred_preprocess_internal

In [ ]:
import intake
col_url = "/home/mpim/m300524/intake-esm-datastore/catalogs/mistral-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [ ]:
# load 2 members for 2 inits from one model
query = dict(experiment_id=[
    'dcppA-hindcast'], table_id='Amon', member_id=['r1i1p1f1', 'r2i1p1f1'], dcpp_init_year=[1970, 1971], variable_id='tas', source_id='MPI-ESM1-2-HR')
cat = col.search(**query)
cdf_kwargs = {'chunks': {'time': 12}, 'decode_times': False}

In [ ]:
def preprocess(ds):
    # extract tiny spatial and temporal subset
    ds = ds.isel(lon=[50, 51, 52], lat=[50, 51, 52],
                 time=np.arange(12 * 2))
    # make time dim identical
    ds = climpred_preprocess_internal(ds)
    return ds

In [ ]:
dset_dict = cat.to_dataset_dict(
    cdf_kwargs=cdf_kwargs, preprocess=preprocess)
# get first dict value
ds = dset_dict[list(dset_dict.keys())[0]]
ds.coords

In [ ]:
ds = climpred_preprocess_post(ds)
ds.coords